In [1]:
#Импортируем библиотеки
import numpy as np
import pandas as pd
import copy
import re
import openpyxl

In [2]:
df = pd.read_csv("C:/Users/NightMare/Desktop/YandexHack/DATA_Pretrained/2021_обработанный_2.csv", encoding='Windows-1251')

In [3]:
df.head(15)

,Направление расходов,Наименование вида расходов,Вид объекта,Дата проведения,Конечный остаток
0,"Мордовцева, 6",ЭЭ,Здание,13.01.2022,13920.02
1,"Пушкинская, 8б",ЭЭ,Здание,19.01.2021,226.79
2,"Пушкинская, 8б",ЭЭ,Здание,20.02.2021,161.12
3,"Пушкинская, 8б",ЭЭ,Здание,22.03.2021,115.67
4,"Пушкинская, 8б",ЭЭ,Здание,21.04.2021,126.38
5,"Пушкинская, 8б",ЭЭ,Здание,20.05.2021,341.56
6,"Пушкинская, 8б",ЭЭ,Здание,22.06.2021,63.29
7,"Пушкинская, 8б",ЭЭ,Здание,21.07.2021,121.25
8,"Пушкинская, 8б",ЭЭ,Здание,23.08.2021,125.06
9,"Пушкинская, 8б",ЭЭ,Здание,23.09.2021,127.08


In [4]:
def remove_whitespace(column):
    return column.str.replace(' ', '')
def remove_suffix(column):
    return column.str.rstrip(',-')

df['Направление расходов'] = remove_whitespace(df['Направление расходов'])
df['Направление расходов'] = remove_suffix(df['Направление расходов'])

In [5]:
def compare_and_assign(df, column_name, replacements):
    for index, value in df[column_name].items():
        for replacement in replacements:
            target_string = replacement['target']
            new_value = replacement['new_value']
            # Удаляем пробелы с обеих сторон строки перед проверкой
            if value.rstrip().endswith(target_string):
                df.at[index, "Вид объекта"] = new_value
                # Удаляем target строку из строки с которой сравнивали
                df.at[index, column_name] = value.replace(target_string, '').strip()
    return df[column_name]  # Возвращаем измененный столбец

# Заменяем значения, оканчивающиеся на "(Буфет)"
replacements = [
    {'target': ',кв.30', 'new_value': ''}
]
df['Направление расходов'] = compare_and_assign(df, 'Направление расходов', replacements)



def compare_and_assign_prefix(df, column_name, replacements):
    for index, value in df[column_name].items():
        for replacement in replacements:
            target_string = replacement['target']
            new_value = replacement['new_value']
            # Удаляем пробелы с обеих сторон строки перед проверкой
            if value.lstrip().startswith(target_string):
                df.at[index, "Вид объекта"] = new_value
                # Удаляем target строку из строки с которой сравнивали
                df.at[index, column_name] = value.replace(target_string, '', 1).strip()
    return df[column_name]  # Возвращаем измененный столбец

replacements = [
    {'target': 'РКЦВилючинск,', 'new_value': ''},
    {'target': 'Благовещенск,', 'new_value': ''},
    {'target': 'Южно-Сахалинск,', 'new_value': ''},
    {'target': 'РКЦКомсомольск-на-Амуре', 'new_value': ''},
    {'target': 'ул,', 'new_value': ''},
    {'target': 'мкр.', 'new_value': ''},
    {'target': 'пр.', 'new_value': ''},
    {'target': 'РКЦ', 'new_value': ''},
    {'target': 'п.Депутатский,мкр.', 'new_value': ''},
    {'target': 'Находка,', 'new_value': ''},
    {'target': 'пер.', 'new_value': ''},
    {'target': 'Ленск', 'new_value': ''},
    {'target': 'Покровск', 'new_value': ''},
    {'target': 'Нерюнгри', 'new_value': ''},
    {'target': 'Южная', 'new_value': ''},
    {'target': '50летСоветскойАрмии', 'new_value': ''},
    {'target': 'Билибино', 'new_value': ''},
    {'target': 'Комсомольск-на-Амуре', 'new_value': ''},
    {'target': ',', 'new_value': ''},
    {'target': ' Профсоюзная,д.,12', 'new_value': ''},
    {'target': 'Зея,пер.2', 'new_value': ''},
    {'target': 'кв.29', 'new_value': ''},
    {'target': 'КарлаМаркса,,,д.16', 'new_value': ''},
    {'target': 'Горького7', 'new_value': ''},
    {'target': 'Биробиджан,', 'new_value': ''},
    {'target': 'Дальнегорск,', 'new_value': ''}
]
df['Направление расходов'] = compare_and_assign_prefix(df, 'Направление расходов', replacements)


def drop_rows_with_empty_values(df):
    # Удаляем строки, содержащие хотя бы одно из значений: 0, NaN, пустую строку или пробелы
    df = df.dropna().replace({'': None, ' ': None}).dropna()
    return df

# Удаляем строки с пустыми значениями
df= drop_rows_with_empty_values(df)



def add_comma_before_d(column):
    # Применяем функцию к каждой ячейке столбца
    column_with_comma = column.apply(lambda x: x[:x.find('д.')] + ',' + x[x.find('д.'):] if 'д.' in x else x)
    
    return column_with_comma

df['Направление расходов'] = add_comma_before_d(df['Направление расходов'])


# Выведите уникальные значения
for value in df['Направление расходов'].unique():
    print(value)

Мордовцева,6
Пушкинская,8б
Седова,32
Фонтанная,19
Семеновская,29
Калинина,261
Океанскийпр-т,34
Светланская,71
Светланская,69
Светланская,111
Светланская,73
Светланская,73А
Ленина,64
Пушкина,59
Океанскийпр-т34
Светланская,73а
Некрасова,102
Пушкина,8б
Дежнева,7
Колхозная32Б
п.Эгвекинот
Дежнева7
пгт.Эгвекинот
Чапаева,5а
пр-т60-летияСССР,5
Миллера,1а,корпус1
Миллера,1а,корпус2
Чапаева,5А
Зеяпер.Жуковский,15
Б.Хмельницкого,52/2
КраснаяПресня,22
Зея,пер.Жуковский,15
п-кт.К.Маркса,36-20,кв.29
Пушкина,4
Пушкина,6
Магадан,п-кт.КарлаМаркса,,д.36/20,кв.29
Пушкина,,д.4
пр-тК.Маркса,29/2
Ленинская,22
Ленинская,33
Ленина,3
М.Амурского,21
М.Амурского,42
Воронежская,154
Калинина,156
Советская,26
Калинина,95
Торговая,54
М-Амурского,21
М-Амурского,42
КарлаМаркса,,д.16
Вилюйск
Кирова,17
Ленина,22
Якутск
п.Усть-Нера
Кирова17
Октябрьская,22
Верхнепортовая,44


In [6]:
df.head()

,Направление расходов,Наименование вида расходов,Вид объекта,Дата проведения,Конечный остаток
0,"Мордовцева,6",ЭЭ,Здание,13.01.2022,13920.02
1,"Пушкинская,8б",ЭЭ,Здание,19.01.2021,226.79
2,"Пушкинская,8б",ЭЭ,Здание,20.02.2021,161.12
3,"Пушкинская,8б",ЭЭ,Здание,22.03.2021,115.67
4,"Пушкинская,8б",ЭЭ,Здание,21.04.2021,126.38


In [7]:
def split_date_column(df, date_column_name):
    df[date_column_name] = pd.to_datetime(df[date_column_name], format='%d.%m.%Y')
    df['Год проведения'] = df[date_column_name].dt.year
    df['Месяц проведения'] = df[date_column_name].dt.month
    df.drop(columns=[date_column_name], inplace=True)
    return df

df = split_date_column(df, 'Дата проведения')

df.head(15)

,Направление расходов,Наименование вида расходов,Вид объекта,Конечный остаток,Год проведения,Месяц проведения
0,"Мордовцева,6",ЭЭ,Здание,13920.02,2022,1
1,"Пушкинская,8б",ЭЭ,Здание,226.79,2021,1
2,"Пушкинская,8б",ЭЭ,Здание,161.12,2021,2
3,"Пушкинская,8б",ЭЭ,Здание,115.67,2021,3
4,"Пушкинская,8б",ЭЭ,Здание,126.38,2021,4
5,"Пушкинская,8б",ЭЭ,Здание,341.56,2021,5
6,"Пушкинская,8б",ЭЭ,Здание,63.29,2021,6
7,"Пушкинская,8б",ЭЭ,Здание,121.25,2021,7
8,"Пушкинская,8б",ЭЭ,Здание,125.06,2021,8
9,"Пушкинская,8б",ЭЭ,Здание,127.08,2021,9


In [8]:
# Переносим Конечный остаток вправо.
new_order = [col for col in df.columns if col != 'Конечный остаток'] + ['Конечный остаток']
df = df[new_order]

In [9]:
# Группировка по другим столбцам и суммирование 'конечный остаток'
df = df.groupby(['Направление расходов', 'Наименование вида расходов', 'Год проведения', 'Месяц проведения'])['Конечный остаток'].sum().reset_index()

In [10]:
df.head()

,Направление расходов,Наименование вида расходов,Год проведения,Месяц проведения,Конечный остаток
0,"Б.Хмельницкого,52/2",ВО,2021,1,2.67
1,"Б.Хмельницкого,52/2",ВО,2021,3,6.14
2,"Б.Хмельницкого,52/2",ВО,2021,4,4.00
3,"Б.Хмельницкого,52/2",ВО,2021,5,4.00
4,"Б.Хмельницкого,52/2",ВО,2021,6,4.00


In [11]:
len(df)

1605

In [12]:
# Сохранение файла в формате CSV с указанием кодировки UTF-8
df.to_csv('C:/Users/NightMare/Desktop/YandexHack/DATA_well_done/2021_welldone.csv', index=False, encoding='Windows-1251')

# Сохранение в Excel
df.to_excel('C:/Users/NightMare/Desktop/YandexHack/DATA_well_done/2021_welldone.xlsx', index=False)